# Projeto 1 - Ciência dos Dados

Nome: Matheus Barros

Nome: Giulia Sampaio

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [105]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import math

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [106]:
import os

filename = 'mcdonalds.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo mcdonalds.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [107]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  Categorias
0  por isso eu prefiro o @mcdonalds_br https://t....           3
1  queria comer uma besteirinha, tipo um big tast...           3
2  @sergiosepsep @mcdonalds_br @ifood eu já tive ...           2
3                 @mvmarizinha @mcdonalds_br mentira           0
4  fui no @mcdonalds_br do @rapososhopping e fiz ...           4

In [108]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  Categorias
0  @mcdonalds_br tenho que me sujeitar a essas co...           4
1                      pq o @mcdonalds_br  e tão bom           3
2            @mcdonalds_br cadê o lanche do travis ?           2
3  @laiz_izabeli @mcdonalds_br da o combo todo po...           3
4  @jonitd_ @mcdonalds_br isso era pra ser um che...           3

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

**Analisar a opinião dos usuários a respeito da rede de fastfood mcdonald's, levando em conta as experiências do usuário.**

* (0) Muito Irrelevante - Um tweet que não reflete nada a respeito da opinião do usuário em relação ao mcdonalds.
 
* (1) Irrelevante - Um tweet que felita a opinião do usuário, mas pouco.

* (2) Neutro - Um tweet que não tenha defeitos nem qualidades, ou estão em equilíbrio.
 
* (3) Relevante - Tem uma opinião, boa ou ruim.

* (4) Muito Relevante - Tem uma opinião que reflita muito a imagem do Mcdonalds, boa ou ruim.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [109]:
train.Categorias = train.Categorias.astype('category')
test.Categorias = test.Categorias.astype('category')

train.Categorias.cat.categories = ['Muito Irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito Relevante']
test.Categorias.cat.categories = ['Muito Irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito Relevante']

train.head()

Treinamento         Categorias
0  por isso eu prefiro o @mcdonalds_br https://t....          Relevante
1  queria comer uma besteirinha, tipo um big tast...          Relevante
2  @sergiosepsep @mcdonalds_br @ifood eu já tive ...             Neutro
3                 @mvmarizinha @mcdonalds_br mentira  Muito Irrelevante
4  fui no @mcdonalds_br do @rapososhopping e fiz ...    Muito Relevante

In [110]:
train.Categorias.value_counts()

Muito Irrelevante    373
Relevante            139
Muito Relevante       89
Neutro                89
Irrelevante           60
Name: Categorias, dtype: int64

In [111]:
train.Categorias.value_counts(True)

Muito Irrelevante    0.497333
Relevante            0.185333
Muito Relevante      0.118667
Neutro               0.118667
Irrelevante          0.080000
Name: Categorias, dtype: float64

In [112]:
def cleanup(text):

    """
    Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """

    #import string

    punctuation = '[!-.:?\;_$%,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    link_sub = re.sub(r'http[^\s]*', '', text_subbed)
    #@[^\s]*

    return link_sub

In [113]:
train_cleanup = []

for row in train.Treinamento:
    row = cleanup(row)
    train_cleanup.append(row)
    
train_cleanup

['por isso eu prefiro o @mcdonaldsbr ',
 'queria comer uma besteirinha tipo um big tasty do @mcdonaldsbr  🤤',
 '@sergiosepsep @mcdonaldsbr @ifood eu já tive problemas com o pedido do mc mas foi pelo ubereats liguei no ubereats fiz a reclamação e eles estornaram o valor no meu cartão tudo isso em menos de 5 min 🙏🏻 ninguém merece ficar puto com o pedido qdo chega e ainda levar canseira depois disso',
 '@mvmarizinha @mcdonaldsbr mentira',
 'fui no @mcdonaldsbr do @rapososhopping e fiz meu pedido quando pedi guardanapo a gerente isso mesmo extremamente grossa virou as costas e me deixou falando até nunca mais',
 'rt @thia6o0 @bolsonarosp @mcdonaldsbr @slpnggiantspt @youtubebrasil sim nunca percebeu as mensagens ocultas na logo ',
 'rt @rap24horasblog combo do travis scott no mcdonald’s faz enorme sucesso e esgota ingredientes do fast food  via @…',
 '@mvmarizinha @mcdonaldsbr sério kkkkkkkkkkkkkkkkkk',
 '@mcdonaldsbr @jamesdeliverybr favor voltem com  o refrigerante em lata que saco toda v

In [114]:
train_text_raw = ' '.join(train_cleanup)
train_text_raw

'por isso eu prefiro o @mcdonaldsbr  queria comer uma besteirinha tipo um big tasty do @mcdonaldsbr  🤤 @sergiosepsep @mcdonaldsbr @ifood eu já tive problemas com o pedido do mc mas foi pelo ubereats liguei no ubereats fiz a reclamação e eles estornaram o valor no meu cartão tudo isso em menos de 5 min 🙏🏻 ninguém merece ficar puto com o pedido qdo chega e ainda levar canseira depois disso @mvmarizinha @mcdonaldsbr mentira fui no @mcdonaldsbr do @rapososhopping e fiz meu pedido quando pedi guardanapo a gerente isso mesmo extremamente grossa virou as costas e me deixou falando até nunca mais rt @thia6o0 @bolsonarosp @mcdonaldsbr @slpnggiantspt @youtubebrasil sim nunca percebeu as mensagens ocultas na logo  rt @rap24horasblog combo do travis scott no mcdonald’s faz enorme sucesso e esgota ingredientes do fast food  via @… @mvmarizinha @mcdonaldsbr sério kkkkkkkkkkkkkkkkkk @mcdonaldsbr @jamesdeliverybr favor voltem com  o refrigerante em lata que saco toda vez vem derramado metade do refrig

In [115]:
todas_palavras_train = train_text_raw.split()
todas_palavras_train

['por',
 'isso',
 'eu',
 'prefiro',
 'o',
 '@mcdonaldsbr',
 'queria',
 'comer',
 'uma',
 'besteirinha',
 'tipo',
 'um',
 'big',
 'tasty',
 'do',
 '@mcdonaldsbr',
 '🤤',
 '@sergiosepsep',
 '@mcdonaldsbr',
 '@ifood',
 'eu',
 'já',
 'tive',
 'problemas',
 'com',
 'o',
 'pedido',
 'do',
 'mc',
 'mas',
 'foi',
 'pelo',
 'ubereats',
 'liguei',
 'no',
 'ubereats',
 'fiz',
 'a',
 'reclamação',
 'e',
 'eles',
 'estornaram',
 'o',
 'valor',
 'no',
 'meu',
 'cartão',
 'tudo',
 'isso',
 'em',
 'menos',
 'de',
 '5',
 'min',
 '🙏🏻',
 'ninguém',
 'merece',
 'ficar',
 'puto',
 'com',
 'o',
 'pedido',
 'qdo',
 'chega',
 'e',
 'ainda',
 'levar',
 'canseira',
 'depois',
 'disso',
 '@mvmarizinha',
 '@mcdonaldsbr',
 'mentira',
 'fui',
 'no',
 '@mcdonaldsbr',
 'do',
 '@rapososhopping',
 'e',
 'fiz',
 'meu',
 'pedido',
 'quando',
 'pedi',
 'guardanapo',
 'a',
 'gerente',
 'isso',
 'mesmo',
 'extremamente',
 'grossa',
 'virou',
 'as',
 'costas',
 'e',
 'me',
 'deixou',
 'falando',
 'até',
 'nunca',
 'mais',
 'r

In [116]:
lista_palavras_train = pd.Series(todas_palavras_train)  
tabela_palavras_train_relativa = lista_palavras_train.value_counts(True)
tabela_palavras_train_relativa

@mcdonaldsbr    0.052298
o               0.031876
e               0.023738
que             0.020723
de              0.020497
                  ...   
email           0.000075
obscuro         0.000075
muitas          0.000075
acéfalo         0.000075
razão           0.000075
Length: 3297, dtype: float64

In [117]:
tabela_palavras_train_relativa.sum()

1.0

In [118]:
# probFraseDado0 = 1
# probFraseDado1 = 1
# probFraseDado2 = 1
# probFraseDado3 = 1
# probFraseDado4 = 1

# for i in frase:
#     probFraseDado0 *= tabela_palavras_teste_relativa[i]
#     probFraseDado1 *= tabela_palavras_teste_relativa[i]
#     probFraseDado2 *= tabela_palavras_teste_relativa[i]
#     probFraseDado3 *= tabela_palavras_teste_relativa[i]
#     probFraseDado4 *= tabela_palavras_teste_relativa[i]

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**